In [1]:
import os
os.chdir('/lustre/scratch117/cellgen/team297/bs16/current_projects/kidney_glomTI_response/code')
import anndata as ad
import scanpy as sc
import os
import pandas as pd
import numpy as np
import useful_functions as uf
sc.set_figure_params(figsize=(6, 6))

import sys
print(sys.executable)


/home/jovyan/my-conda-envs/myenv/bin/python


In [2]:
import warnings
warnings.filterwarnings('ignore')
#this will hide the awful pandas deprec warnings that are currently plaguing scanpy

#now print versions
print(sc.__version__)
print(ad.__version__)
print(pd.__version__)
os.chdir('/lustre/scratch117/cellgen/team297/bs16/current_projects/kidney_glomTI_response')


1.9.1
0.8.0
1.4.3


In [3]:
import scipy.io as spio
kuppe_counts = spio.mmread('./data/public_data/kuppe_2021/CD10negative/kidneyMap_UMI_counts.mtx')
from scipy import sparse
kuppe_counts = sparse.csr_matrix(kuppe_counts).T

In [4]:
adata = ad.AnnData(X=kuppe_counts)

In [5]:
adata

AnnData object with n_obs × n_vars = 51849 × 23446

In [6]:
kuppe_rows = pd.read_csv("./data/public_data/kuppe_2021/CD10negative/kidneyMap_UMI_counts_rowData.txt")

In [7]:
adata.var_names = np.array(kuppe_rows['Gene.Symbol'])
adata.var['Symbol'] = np.array(kuppe_rows['Gene.Symbol'])
adata.var['ID'] = np.array(kuppe_rows['ENSEMBL.ID'])

In [8]:
kuppe_cols = pd.read_csv("./data/public_data/kuppe_2021/CD10negative/kidneyMap_UMI_counts_colData.txt")

In [9]:
kuppe_cols['Annotation.Level.2'].value_counts()

Vasa Recta                         18804
Glomerular Capillaries              4377
B Cells                             3101
Macrophages                         3030
Venular Endothelium                 2724
Injured Endothelium                 2499
Thick Ascending Limb                2266
Intercalated Cells                  1395
Distal Convoluted Tubule            1342
Fibroblast                          1215
Monocytes                           1111
Proximal Tubule                      917
Injured Tubule                       909
Arterioral Endothelium               901
Dendritic Cells                      841
Myofibroblast                        831
Descending Thin Limb                 806
Natural Killer Cells                 740
Connecting Tubule                    662
Pericytes                            643
Lymph Endothelium                    509
T Cells                              484
Smooth Muscle Cells                  426
Uroethlial Cells                     246
Plasma Cells    

In [10]:
adata.obs['celltype'] = np.array(kuppe_cols['Annotation.Level.2']) 
adata.obs['broad_celltype'] = np.array(kuppe_cols['Annotation.Level.1']) 
adata.obs['Experiment'] = np.array(kuppe_cols['Patient ID'])
adata.obs['Dataset'] = 'Kuppe'

In [11]:
#subset to some of the samples
ckd_samples = ["CDm8", "CDm9", "CDm10"]
healthy_samples = ["CDm1", "CDp1", "CDm3", "CDm5"]
adata = adata[adata.obs['Experiment'].isin(healthy_samples)]

In [12]:
adata.var_names_make_unique()

In [13]:
adata.layers['counts'] = adata.X

In [14]:
adata

AnnData object with n_obs × n_vars = 19491 × 23446
    obs: 'celltype', 'broad_celltype', 'Experiment', 'Dataset'
    var: 'Symbol', 'ID'
    layers: 'counts'

In [15]:
#save this
adata.write_h5ad('./data/public_data/kuppe_2021/kuppe_adata.h5ad') 